In [ ]:
from openai import OpenAI

API_KEY="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"
model="codellama-34b-instruct"

In [ ]:
def LLM_CodeGenerator(mode, user_prompt, num_loops, previous_prompts_responses=None):
    if previous_prompts_responses is None:
        previous_prompts_responses = []

    for i in range(num_loops):
        print("*****\n\n\n")
        print(f"\nModel {mode} Iteration {i+1}:")
        if mode == "Starter":
            gprompt,generated_code = generate_initial_code_based_on_prompt(user_prompt, previous_prompts_responses)
        elif mode == "Debug":
            gprompt,generated_code = analyze_code_for_errors(previous_prompts_responses)
        elif mode == "Optimize":
            gprompt,generated_code = optimize_code(previous_prompts_responses)
        elif mode == "Build test":
            gprompt,generated_code = build_tester_code(previous_prompts_responses)
        elif mode == "Compile test":
            gprompt = "Compile and test"
            generated_code, response_message = compile_and_run_tests(previous_prompts_responses)
            print(response_message)
        else:
            return "Invalid mode selected"
        print("*****\n\n\n")
        

        print(generated_code)
        previous_prompts_responses.append({"role": "user", "content": gprompt})
        previous_prompts_responses.append({"role": "assistant", "content": generated_code})

        # Self-diagnosis prompts
        self_diagnosis_prompts = [
            "Identify potential errors in the generated code",
            "Suggest ways to make the generated code more efficient",
            "Provide any additional improvements or best practices for the generated code"
        ]

        if mode != "Compile test" : 
            extracted_code = extract_generated_code_from_chat_log(previous_prompts_responses[-1]["content"])
        else:
            extracted_code = generated_code
        
        if mode == "Starter":
            for prompt in self_diagnosis_prompts:
                prompt1 = prompt +  ". Then output revised code to reflect this feedback.\n\n"
    #            self_diagnosis_response = call_pplx_api(prompt1, previous_prompts_responses)
                prompt1 += "Original user prompt:"
                prompt1 += user_prompt
                prompt1 += "Generated code:"
                prompt1 += extracted_code
                prompt1 += "Self-diagnosis response:<Enter response here>\n\n"
                prompt1 += "Revised code:"

                self_diagnosis_response = call_pplx_api(prompt1, [])
                print(f"\n{prompt1}")
                print("LLM response:\n***********\n")
                print(self_diagnosis_response)
                previous_prompts_responses.append({"role": "user", "content": prompt1})
                previous_prompts_responses.append({"role": "assistant", "content": self_diagnosis_response})
                extracted_code = extract_generated_code_from_chat_log(self_diagnosis_response)

    return extracted_code, previous_prompts_responses

def generate_initial_code_based_on_prompt(user_prompt, previous_prompts_responses):
    prompt = f"Write code based on the following prompt: {user_prompt}"
    return prompt, call_pplx_api(prompt, previous_prompts_responses)

def analyze_code_for_errors(previous_prompts_responses):
    prompt = "Analyze the previously generated code for errors and provide corrections"
    code_to_analyze = previous_prompts_responses[-1]["content"]
    prompt += f"\n\nCode to analyze:\n{code_to_analyze}"
    return prompt, call_pplx_api(prompt, previous_prompts_responses)

def optimize_code(previous_prompts_responses):
    prompt = "Optimize the previously generated code for performance and efficiency"
    code_to_optimize = previous_prompts_responses[-1]["content"]
    prompt += f"\n\nCode to optimize:\n{code_to_optimize}"
    return prompt,call_pplx_api(prompt, previous_prompts_responses)

def build_tester_code(previous_prompts_responses):
    prompt = "Build tester code for the previously generated code to ensure its correctness"
    code_to_test = previous_prompts_responses[-1]["content"]
    prompt += f"\n\nCode to build tester for:\n{code_to_test}"
    return prompt,call_pplx_api(prompt, previous_prompts_responses)


def extract_generated_code_from_chat_log(text_):
    prompt = """Return only the last block of valid python code from this chat session log. Only output syntactically correct code. No explanations or details. Remove any leading or trailing whitespace or quotes.
    Example:
    Input log:
    User: Write code to add two numbers. Then write code to multiply two numbers. Then write code to subtract two numbers. Then test the add and multiply functions.
    def add(a, b):
        return a + b
    def subtract(a, b):
        return a - b
    def multiply(a, b):
        return a * b
    add(1, 2)
    multiply(3, 4)
    Extracted Code:
    def add(a, b):
        return a + b
    def subtract(a, b):
        return a - b
    def multiply(a, b):
        return a * b
    add(1, 2)
    multiply(3, 4)

    Input log:    
    """
    prompt += f"\n\n:\n{text_}"
    prompt += "\n\nExtracted Code:"
    return  call_pplx_api(prompt, [])
    


def compile_and_run_tests(previous_prompts_responses):
#    prompt = "Compile the previously generated code and run tests to verify its functionality"
    
    extracted_code = extract_generated_code_from_chat_log(previous_prompts_responses[-1]["content"])


    #prompt = "Return only the final code from this chat session log. Only output syntactically correct code. No explanations or details"
    #code_to_compile_and_test = previous_prompts_responses[-1]["content"]

    #prompt += f"\n\n:\n{code_to_compile_and_test}"
#    resp =  call_pplx_api(prompt, extracted_code)
 #   code_to_compile_and_test = resp

    # Specify the path to the Python file you want to compile
    source_file = "example.py"

    # write the code to a file
    with open(source_file, "w") as file:
        file.write(extracted_code)

    # Compile the Python file
    response = ""
    try:
        #py_compile.compile(source_file)
        py_compile.compile(source_file, doraise=True)

        response = f"Successfully compiled {source_file}"
    except py_compile.PyCompileError as e:
        response = f"Compilation failed: {e}"

    return extracted_code, response.strip()


 #       prompt += f"\n\nCode to compile and test:\n{code_to_compile_and_test}"
 #       return call_pplx_api(prompt, previous_prompts_responses)

def call_pplx_api(prompt, previous_prompts_responses=None):
    print("  ____  ")
    print(" / __ \\ ")
    print("/ /  \\ \\")
    print("\\ \\__/ /")
    print(" \\____/ ")
    print("\n")

    model = "codellama-34b-instruct"
    
    messages = [
        {
            "role": "system",
            "content": (
                "You're a super powerful code copilot capable of running loops. You are top coder at Google. You always output python code and decline to output code in any other language."
            ),
        }
    ]

    if previous_prompts_responses:
        messages.extend(previous_prompts_responses)

    messages.append({
        "role": "user",
        "content": prompt,
    })

    client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )

    return response.choices[0].message.content

previous_prompts_responses = []


# Main program
while True:
    print("New code generation iteration\n")
    ## print ascii art for B
    print("  ____  ")
    print(" / __ \\ ")
    print("/ /  \\ \\")
    print("\\ \\__/ /")
    print(" \\____/ ")
    print("  ____  ")
    print(" / __ \\ ")
    print("/ /  \\ \\")
    print("\\ \\__/ /")
    print(" \\____/ ")
    print("\n")
    

    user_input = ""
    selected_mode = input("Enter the mode (Starter/Debug/Optimize/Build test/Compile test/Exit): ")
    if selected_mode == "Exit":
        break
    elif selected_mode not in ["Starter", "Debug", "Optimize", "Build test", "Compile test"]:
        print("Invalid mode selected")
        continue
    elif selected_mode == "Starter" : 
        user_input = input("Enter the user prompt: ")

    num_loops = int(input("Enter the number of loops for each step: "))
    generated_code, previous_prompts_responses = LLM_CodeGenerator(selected_mode, user_input, num_loops, previous_prompts_responses=previous_prompts_responses)
    print("\nFinal generated code:")
    print(generated_code)